# Import the libraries

In [28]:

#from PyUncertainNumber.UC.uncertainNumber import UncertainNumber
#from PyUncertainNumber.UC.utils import UNEncoder, cd_root_dir

import numpy as np

import importlib
#import PyUncertainNumber.UP.local_optimisation as up
#from PyUncertainNumber.UP.main import up_bb
from PyUncertainNumber.UP.local_optimisation import local_optimisation_method 
import PyUncertainNumber.UP.main as main
#importlib.reload(up)
importlib.reload(main)



<module 'PyUncertainNumber.UP.main' from 'C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\src\\PyUncertainNumber\\UP\\main.py'>

# Problem setup

The numerical example models a simple cantilever beam with length, L, distance to the neutral axis y, Young’s modulus, E, second moment of inertia, I, and load, F. We will compute the stress σ, and the displacement d, in the beam assumign the above input parameters are uncertain numbers.

We will use this example throughout the document to illustrate the differences among uncertainty quantification approaches.

![alt text](cantilever.png)

Fig.1. Cantilever beam with input parameters.  

# Create the function

A function is created which calculates the stress, σ, as a function of L, y, E, I, and F.
- The function's input is only the uncertain numbers.
- As sampling techniques are used it is likely that for a certain input combination the airfoil will fail to reach a solution. we use the try function to accomodate for this. 

In [29]:
def cantilever_beam_deflection(x):
    """Calculates deflection and stress for a cantilever beam.

    Args:
        x (np.array): Array of input parameters:
            x[0]: Length of the beam (m)
            x[1]: Second moment of area (mm^4)
            x[2]: Applied force (N)
            x[3]: Young's modulus (MPa)

    Returns:
        float: deflection (m)
               Returns np.nan if calculation error occurs.
    """

    beam_length = x[0]
    I = x[1]
    F = x[2]
    E = x[3]
    try:  # try is used to account for cases where the input combinations leads to error in fun due to bugs
        deflection = F * beam_length**3 / (3 * E * 10**6 * I)  # deflection in m
        
    except:
        deflection = np.nan

    return deflection




## Test the function

To ensure that the function yields meaningful results. We consider that input has the nominal values seen below.

The function should yield deflection equal to 0.162m.
 

In [30]:
# test the function

y  = 0.155 # m
L  = 10.05 # m
I = 0.000386 # m**4
F = 37 # kN
E = 200 # GPa

x = np.array([L, I, F, E])
deflection = cantilever_beam_deflection(x)

print(deflection) # 0.162m

0.1621665786917099


# Propagate epistemic uncertainty 



## Uncertainty Characterisation

Create 5 uncertain numbers assuming they are intervals with the lower and upper values are seen below

y = [0.145, 0.155] m

L = [9.95, 10.05] m

I = [0.0003861591, 0.0005213425] m**4

F = [11, 37] kN

E = [200, 220] GPa
 

In [ ]:
# %% Inputs
#Define input intervals

#y = np.array([0.145, 0.155]) #m

L =  np.array([9.95, 10.05]) # m

I =  np.array([0.0003861591, 0.0005213425])# m**4

F =  np.array([11, 37]) # kN

E =  np.array([200, 220]) # GPa

# Create a 2D np.array with all uncertain input parameters in the **correct** order.
xInt = np.array([L, I, F, E])
print(xInt)

[[9.950000e+00 1.005000e+01]
 [3.861591e-04 5.213425e-04]
 [1.100000e+01 3.700000e+01]
 [2.000000e+02 2.200000e+02]]


## Uncertainty Propagation methods 

Choose multiple black box techniques to propagate the intervals through the model. 

### Endpoint propagation

The endpoint propagation method (Dong and Shah, 1987) is a straightforward way to project intervals through the code, by projecting all input combinations produced by the Cartesian product of the interval bounds. This results in a total of n = 2**d. 

For the working example, there are d = 5 intervals which results in n = 2**5 = 32 input combinations.

#### Assumptions




In [32]:
# How many input combinations are expected from the endpoint propagation?
d = xInt.shape[0] # is the number of uncertain input expressed as intervals.
n = 2**d # The total number of combinations 
print("Total number of input combinations for the endpoint method:", n) 

Total number of input combinations for the endpoint method: 16


In [ ]:
method = "endpoint"
base_path = "C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\cantilever_beam"
min_y, max_y, x_miny, x_maxy = main.up_bb(xInt, cantilever_beam_deflection, n = None, method= method, save_raw_data = "no", base_path = base_path)

#TODO envelope the results of min_y nad max_y 
# plot the extremities for each input variable
print(min_y)
print(max_y)


<class 'numpy.float64'>


100%|██████████| 15/15 [00:00<00:00, 10958.82it/s]

[0.03149161]
[0.16209977]


In [34]:
# The endpoint method is being called 

method = "endpoint"
base_path = "C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\cantilever_beam"
a = main.up_bb(xInt, cantilever_beam_deflection, n = None, method= method, save_raw_data="yes", base_path = base_path)
              

#min_y, max_y, x_miny, x_maxy = up.endpoints_method(xInt, cantilever_beam_deflection, save_raw_data = 'no')

# check the results.
# print(min_y)
# print(max_y)
# print(x_miny)
# print(x_maxy)


100%|██████████| 15/15 [00:00<?, ?it/s]

There are no NA values produced by the input


In [35]:
method = "montecarlo"
base_path = "C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\cantilever_beam"
a = main.up_bb(xInt, cantilever_beam_deflection, n = 10, method= method, save_raw_data="no", base_path = base_path)
              

#min_y, max_y, x_miny, x_maxy = up.endpoints_method(xInt, cantilever_beam_deflection, save_raw_data = 'no')

# check the results.
# print(min_y)
# print(max_y)
# print(x_miny)
# print(x_maxy)


100%|██████████| 9/9 [00:00<?, ?it/s]


In [36]:
print(a)

(array([0.03628297]), array([0.13800991]), array([[1.00387387e+01, 5.19537641e-04, 1.22817906e+01, 2.19714154e+02]]), array([[1.00387734e+01, 4.36176576e-04, 3.67322999e+01, 2.05776705e+02]]))


In [42]:
method = "local_optimisation"
#base_path = "C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\cantilever_beam"
a = main.up_bb(xInt, cantilever_beam_deflection, x0 = None, method = method, method_loc = 'Nelder-Mead')
              
#print(a)

The intermediate steps cannot be saved for local optimisation
minimisation: Optimization terminated successfully.
maximisation: Optimization terminated successfully.


In [38]:
# The local_optimisation is being called 
a = local_optimisation_method(xInt, cantilever_beam_deflection, x0 = None, method_loc = 'Nelder-Mead')

# check the results.
print(a)

(np.float64(-0.16209976503208137), np.float64(0.031491609802257306), array([1.005000e+01, 3.861591e-04, 3.700000e+01, 2.000000e+02]), array([9.950000e+00, 5.213425e-04, 1.100000e+01, 2.200000e+02]), 'Optimization terminated successfully.', 'Optimization terminated successfully.', 26, 26, 41, 40, (array([[1.005000e+01, 3.861591e-04, 3.700000e+01, 2.000000e+02],
       [1.005000e+01, 3.861591e-04, 3.700000e+01, 2.000000e+02],
       [1.005000e+01, 3.861591e-04, 3.700000e+01, 2.000000e+02],
       [1.005000e+01, 3.861591e-04, 3.700000e+01, 2.000000e+02],
       [1.005000e+01, 3.861591e-04, 3.700000e+01, 2.000000e+02]]), array([-0.16209977, -0.16209977, -0.16209977, -0.16209977, -0.16209977])), (array([[9.950000e+00, 5.213425e-04, 1.100000e+01, 2.200000e+02],
       [9.950000e+00, 5.213425e-04, 1.100000e+01, 2.200000e+02],
       [9.950000e+00, 5.213425e-04, 1.100000e+01, 2.200000e+02],
       [9.950000e+00, 5.213425e-04, 1.100000e+01, 2.200000e+02],
       [9.950000e+00, 5.213425e-04, 1.1

### Subinterval reconstitution propagation

The input intervals are partitioned into smaller intervals, which are then propagated through the model using endpoint propagation and the output interval can be reassembled (Ferson and Hajagos, 2004).

In [39]:
# How many input combinations are expected from the subinterval reconstitution propagation?
d = 5 # is the number of uncertain input expressed as intervals.
m = 4 # is the number of partitions. 
n = (m+1)**d # The total number of combinations 
print("Total number of input combinations for the subinterval reconstitution method:", n) 

Total number of input combinations for the subinterval reconstitution method: 3125


### Sampling propagation

- Brute Monte Carlo
- Latin Hypercube

In [40]:
# Check the results

### Optimisation

- Local optimisation
- Genetic algorithm
